In [1]:
import torch
import torch.nn as nn


class Backbone(nn.Module):
    """Base class for backbone networks. Handles freezing layers etc.
    args:
        frozen_layers  -  Name of layers to freeze. Either list of strings, 'none' or 'all'. Default: 'none'.
    """
    def __init__(self, frozen_layers=()):
        super().__init__()

        if isinstance(frozen_layers, str):
            if frozen_layers.lower() == 'none':
                frozen_layers = ()
            elif frozen_layers.lower() != 'all':
                raise ValueError('Unknown option for frozen layers: \"{}\". Should be \"all\", \"none\" or list of layer names.'.format(frozen_layers))

        self.frozen_layers = frozen_layers
        self._is_frozen_nograd = False


    def train(self, mode=True):
        super().train(mode)
        if mode == True:
            self._set_frozen_to_eval()
        if not self._is_frozen_nograd:
            self._set_frozen_to_nograd()
            self._is_frozen_nograd = True
        return self


    def _set_frozen_to_eval(self):
        if isinstance(self.frozen_layers, str) and self.frozen_layers.lower() == 'all':
            self.eval()
        else:
            for layer in self.frozen_layers:
                getattr(self, layer).eval()


    def _set_frozen_to_nograd(self):
        if isinstance(self.frozen_layers, str) and self.frozen_layers.lower() == 'all':
            for p in self.parameters():
                p.requires_grad_(False)
        else:
            for layer in self.frozen_layers:
                for p in getattr(self, layer).parameters():
                    p.requires_grad_(False)
import numpy as np
import torch
import torch.nn as nn


class channel_selection(nn.Module):
    """
    Select channels from the output of BatchNorm2d layer. It should be put directly after BatchNorm2d layer.
    The output shape of this layer is determined by the number of 1 in `self.indexes`.
    """
    def __init__(self, num_channels):
        """
        Initialize the `indexes` with all one vector with the length same as the number of channels.
        During pruning, the places in `indexes` which correpond to the channels to be pruned will be set to 0.
        """
        super(channel_selection, self).__init__()
        self.indexes = nn.Parameter(torch.ones(num_channels))

    def forward(self, input_tensor):
        """
        Parameter
        ---------
        input_tensor: (N,C,H,W). It should be the output of BatchNorm2d layer.
        """
        selected_index = np.squeeze(np.argwhere(self.indexes.data.cpu().numpy()))
        if selected_index.size == 1:
            selected_index = np.resize(selected_index, (1,)) 
        output = input_tensor[:, :, :, :]
        return output


In [2]:
import math
import torch.nn as nn
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torchvision.models.resnet import model_urls
# from .base import Backbone
# from .channel_selection import channel_selection



def conv3x3(in_planes, out_planes, stride=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, bias=False, dilation=dilation)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1, use_bn=True):
        super(BasicBlock, self).__init__()
        self.use_bn = use_bn
        self.conv1 = conv3x3(inplanes, planes, stride, dilation=dilation)

        if use_bn:
            self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, dilation=dilation)

        if use_bn:
            self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)

        if self.use_bn:
            out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)

        if self.use_bn:
            out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=dilation, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.select = channel_selection(planes*4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
#         out = self.select(out)
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(Backbone):
    """ ResNet network module. Allows extracting specific feature blocks."""
    def __init__(self, block, layers, output_layers, num_classes=1000, inplanes=64, dilation_factor=1, frozen_layers=()):
        self.inplanes = inplanes
        super(ResNet, self).__init__(frozen_layers=frozen_layers)
        self.output_layers = output_layers
        self.conv1 = nn.Conv2d(3, inplanes, kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        stride = [1 + (dilation_factor < l) for l in (8, 4, 2)]
        self.layer1 = self._make_layer(block, inplanes, layers[0], dilation=max(dilation_factor//8, 1))
        self.layer2 = self._make_layer(block, inplanes*2, layers[1], stride=stride[0], dilation=max(dilation_factor//4, 1))
        self.layer3 = self._make_layer(block, inplanes*4, layers[2], stride=stride[1], dilation=max(dilation_factor//2, 1))
        self.layer4 = self._make_layer(block, inplanes*8, layers[3], stride=stride[2], dilation=dilation_factor)

        out_feature_strides = {'conv1': 4, 'layer1': 4, 'layer2': 4*stride[0], 'layer3': 4*stride[0]*stride[1],
                               'layer4': 4*stride[0]*stride[1]*stride[2]}

        # TODO better way?
        if isinstance(self.layer1[0], BasicBlock):
            out_feature_channels = {'conv1': inplanes, 'layer1': inplanes, 'layer2': inplanes*2, 'layer3': inplanes*4,
                               'layer4': inplanes*8}
        elif isinstance(self.layer1[0], Bottleneck):
            base_num_channels = 4 * inplanes
            out_feature_channels = {'conv1': inplanes, 'layer1': base_num_channels, 'layer2': base_num_channels * 2,
                                    'layer3': base_num_channels * 4, 'layer4': base_num_channels * 8}
        else:
            raise Exception('block not supported')

        self._out_feature_strides = out_feature_strides
        self._out_feature_channels = out_feature_channels

        # self.avgpool = nn.AvgPool2d(7, stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(inplanes*8 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def out_feature_strides(self, layer=None):
        if layer is None:
            return self._out_feature_strides
        else:
            return self._out_feature_strides[layer]

    def out_feature_channels(self, layer=None):
        if layer is None:
            return self._out_feature_channels
        else:
            return self._out_feature_channels[layer]

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
                channel_selection(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, dilation=dilation))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def _add_output_and_check(self, name, x, outputs, output_layers):
        if name in output_layers:
            outputs[name] = x
        return len(output_layers) == len(outputs)

    def forward(self, x, output_layers=None):
        """ Forward pass with input x. The output_layers specify the feature blocks which must be returned """
        outputs = OrderedDict()

        if output_layers is None:
            output_layers = self.output_layers

        x = self.conv1(x)
        x = self.bn1(x) ####### select daal dena
        x = self.relu(x)

        if self._add_output_and_check('conv1', x, outputs, output_layers):
            return outputs

        x = self.maxpool(x)

        x = self.layer1(x)

        if self._add_output_and_check('layer1', x, outputs, output_layers):
            return outputs

        x = self.layer2(x)

        if self._add_output_and_check('layer2', x, outputs, output_layers):
            return outputs

        x = self.layer3(x)

        if self._add_output_and_check('layer3', x, outputs, output_layers):
            return outputs

        x = self.layer4(x)

        if self._add_output_and_check('layer4', x, outputs, output_layers):
            return outputs

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        if self._add_output_and_check('fc', x, outputs, output_layers):
            return outputs

        if len(output_layers) == 1 and output_layers[0] == 'default':
            return x

        raise ValueError('output_layer is wrong.')


def resnet_baby(output_layers=None, pretrained=False, inplanes=16, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, inplanes=inplanes, **kwargs)

    if pretrained:
        raise NotImplementedError
    return model


def resnet18(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, **kwargs)

    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet50(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(Bottleneck, [3, 4, 6, 3], output_layers, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50'], progress = False), strict = False )
    return model

def resnet101(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(Bottleneck,[3, 4, 23, 3], output_layers, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model

In [3]:
model = resnet101()
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (select): channel_selection()
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 

In [4]:
cd /workspace/pytracking

/workspace/pytracking


In [5]:
from ltr.admin.loading import torch_load_legacy

In [6]:
ckpt = torch_load_legacy('/workspace/tracking_datasets/saved_ckpts/ltr/dimp/sparse/dimp101/DiMPnet_ep0050.pth.tar')['net']

from collections import OrderedDict
new_state = OrderedDict()

for key, value in ckpt.items():
    key = key[18:] # remove `module.`
    new_state[key] = value
model.load_state_dict(new_state, strict = False)

_IncompatibleKeys(missing_keys=['layer1.0.select.indexes', 'layer1.0.downsample.2.indexes', 'layer1.1.select.indexes', 'layer1.2.select.indexes', 'layer2.0.select.indexes', 'layer2.0.downsample.2.indexes', 'layer2.1.select.indexes', 'layer2.2.select.indexes', 'layer2.3.select.indexes', 'layer3.0.select.indexes', 'layer3.0.downsample.2.indexes', 'layer3.1.select.indexes', 'layer3.2.select.indexes', 'layer3.3.select.indexes', 'layer3.4.select.indexes', 'layer3.5.select.indexes', 'layer3.6.select.indexes', 'layer3.7.select.indexes', 'layer3.8.select.indexes', 'layer3.9.select.indexes', 'layer3.10.select.indexes', 'layer3.11.select.indexes', 'layer3.12.select.indexes', 'layer3.13.select.indexes', 'layer3.14.select.indexes', 'layer3.15.select.indexes', 'layer3.16.select.indexes', 'layer3.17.select.indexes', 'layer3.18.select.indexes', 'layer3.19.select.indexes', 'layer3.20.select.indexes', 'layer3.21.select.indexes', 'layer3.22.select.indexes', 'layer4.0.select.indexes', 'layer4.0.downsampl

In [ ]:
bn1 = torch.tensor([4,2,1,3])
y,i = torch.sort(bn1)
bn1

In [7]:
total1 = 0
total2 = 0
total3 = 0
for idx, m in model.named_modules():
    if isinstance(m, nn.BatchNorm2d):
        if idx.split('.')[0] == 'layer1':
            total1 += m.weight.data.shape[0]
        if idx.split('.')[0] == 'layer2':
            total2 += m.weight.data.shape[0]
        if idx.split('.')[0] == 'layer3':
            total3 += m.weight.data.shape[0]

# bn1 = torch.zeros(total1)
# bn2 = torch.zeros(total2)
# bn3 = torch.zeros(total3)
# index1 = 0
# index2 = 0
# index3 = 0
# for (idx, m) in model.named_modules():
#     if isinstance(m, nn.BatchNorm2d):
#         if idx.split('.')[0] == 'layer1':
#             size = m.weight.data.shape[0]
#             bn1[index1:(index1+size)] = m.weight.data.abs().clone()
#             index1 += size
#         if idx.split('.')[0] == 'layer2':
#             size = m.weight.data.shape[0]
#             bn2[index2:(index2+size)] = m.weight.data.abs().clone()
#             index2 += size
#         if idx.split('.')[0] == 'layer3':
#             size = m.weight.data.shape[0]
#             bn3[index3:(index3+size)] = m.weight.data.abs().clone()
#             index3 += size
            
# y1, i = torch.sort(bn1)
# thre_index1 = int(total1 * 0.25)
# thre1 = y1[thre_index1]

# y2, i = torch.sort(bn2)
# thre_index2 = int(total2 * 0.25)
# thre2 = y2[thre_index2]

# y3, i = torch.sort(bn3)
# thre_index3 = int(total3 * 0.25)
# thre3 = y3[thre_index3]


pruned = 0
cfg = []
cfg_mask = []
# thre=0.5
modules = list(model.named_modules())
for k, (idx, m) in enumerate(modules):
#     print(idx)
    if isinstance(m, nn.BatchNorm2d) :
        if idx.split('.')[0] == 'layer1':
            weight_copy = m.weight.data.abs().clone()
            y1, i1 = torch.sort(weight_copy)
            thre_index1 = int(len(y1) * 0.25)
            thre1 = y1[thre_index1]
            print('thre:',thre1)
            mask = weight_copy.gt(thre1).float()
            pruned = pruned + mask.shape[0] - torch.sum(mask)
            m.weight.data.mul_(mask)
            m.bias.data.mul_(mask)
#             if not isinstance(modules[k-2],nn.Sequential):
#             print(idx.split('.')[1])
            if not idx.split('.')[2]=='downsample':
                cfg.append(int(torch.sum(mask)))
                print(torch.sum(mask))
            else:
                print('yes')
                print(torch.sum(mask),'**')
    #             print(modules[k-2])
            cfg_mask.append(mask.clone())
            print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.format(k, mask.shape[0], int(torch.sum(mask))))
        elif idx.split('.')[0] == 'layer2':
            weight_copy = m.weight.data.abs().clone()
            y1, i1 = torch.sort(weight_copy)
            thre_index1 = int(len(y1) * 0.25)
            thre1 = y1[thre_index1]
            print('thre:',thre1)
            mask = weight_copy.gt(thre1).float()
            pruned = pruned + mask.shape[0] - torch.sum(mask)
            m.weight.data.mul_(mask)
            m.bias.data.mul_(mask)
#             if not isinstance(modules[k-2],nn.Sequential):
            if not idx.split('.')[2]=='downsample':
                cfg.append(int(torch.sum(mask)))
                print(torch.sum(mask))
            else:
                print('yes')
                print(torch.sum(mask),'**')
    #             print(modules[k-2])
            cfg_mask.append(mask.clone())
            print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.format(k, mask.shape[0], int(torch.sum(mask))))
        elif idx.split('.')[0] == 'layer3':
            weight_copy = m.weight.data.abs().clone()
            y1, i1 = torch.sort(weight_copy)
            thre_index1 = int(len(y1) * 0.5)
            thre1 = y1[thre_index1]
            print('thre:',thre1)
            mask = weight_copy.gt(thre1).float()
            pruned = pruned + mask.shape[0] - torch.sum(mask)
            m.weight.data.mul_(mask)
            m.bias.data.mul_(mask)
            if not idx.split('.')[2]=='downsample':
                cfg.append(int(torch.sum(mask)))
                print(torch.sum(mask))
            else:
                print('yes')
                print(torch.sum(mask),'**')
    #             print(modules[k-2])
            cfg_mask.append(mask.clone())
            print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.format(k, mask.shape[0], int(torch.sum(mask))))
            
        else:
            weight_copy = m.weight.data.abs().clone()
            mask = weight_copy.gt(0.0).float()
#             pruned = pruned + mask.shape[0] - torch.sum(mask)
            m.weight.data.mul_(mask)
            m.bias.data.mul_(mask)
            try:
                if not idx.split('.')[2]=='downsample':
                    cfg.append(int(torch.sum(mask)))
                    print(torch.sum(mask))
                else:
                    print('yes')
                    print(torch.sum(mask),'**')
            except:
                cfg.append(int(torch.sum(mask)))
                print(torch.sum(mask))
    #             print(modules[k-2])
            cfg_mask.append(mask.clone())
            print('layer index: {:d} \t total channel: {:d} \t remaining channel: {:d}'.format(k, mask.shape[0], int(torch.sum(mask))))
    elif isinstance(m, nn.MaxPool2d):
        cfg.append('M')

pruned_ratio = pruned/(total1+total2+total3)

tensor(64.)
layer index: 2 	 total channel: 64 	 remaining channel: 64
thre: tensor(0.1160)
tensor(47.)
layer index: 8 	 total channel: 64 	 remaining channel: 47
thre: tensor(0.1332)
tensor(47.)
layer index: 10 	 total channel: 64 	 remaining channel: 47
thre: tensor(0.0078)
tensor(191.)
layer index: 12 	 total channel: 256 	 remaining channel: 191
thre: tensor(0.1260)
yes
tensor(191.) **
layer index: 17 	 total channel: 256 	 remaining channel: 191
thre: tensor(1.9898e-08)
tensor(47.)
layer index: 21 	 total channel: 64 	 remaining channel: 47
thre: tensor(2.5724e-08)
tensor(47.)
layer index: 23 	 total channel: 64 	 remaining channel: 47
thre: tensor(0.0004)
tensor(191.)
layer index: 25 	 total channel: 256 	 remaining channel: 191
thre: tensor(0.1237)
tensor(47.)
layer index: 30 	 total channel: 64 	 remaining channel: 47
thre: tensor(0.1418)
tensor(47.)
layer index: 32 	 total channel: 64 	 remaining channel: 47
thre: tensor(0.0005)
tensor(191.)
layer index: 34 	 total channel: 25

In [8]:
pruned_ratio

tensor(0.4721)

In [9]:
len(cfg)

101

In [ ]:
y

In [10]:
cfg

[64,
 'M',
 47,
 47,
 191,
 47,
 47,
 191,
 47,
 47,
 191,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 512,
 512,
 2048,
 512,
 512,
 2048,
 512,
 512,
 2048]

In [11]:
len(cfg)

101

In [12]:
# model

In [13]:
import math
import torch.nn as nn
from collections import OrderedDict
import torch.utils.model_zoo as model_zoo
from torchvision.models.resnet import model_urls
# from .base import Backbone
# from .channel_selection import channel_selection



def conv3x3(in_planes, out_planes, stride=1, dilation=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=dilation, bias=False, dilation=dilation)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, dilation=1, use_bn=True):
        super(BasicBlock, self).__init__()
        self.use_bn = use_bn
        self.conv1 = conv3x3(inplanes, planes, stride, dilation=dilation)

        if use_bn:
            self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes, dilation=dilation)

        if use_bn:
            self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)

        if self.use_bn:
            out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)

        if self.use_bn:
            out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, cfg, stride=1, downsample=None, dilation=1,):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes,cfg[1], kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(cfg[1])
        self.conv2 = nn.Conv2d(cfg[1], cfg[2], kernel_size=3, stride=stride,
                               padding=dilation, bias=False, dilation=dilation)
        self.bn2 = nn.BatchNorm2d(cfg[2])
        self.conv3 = nn.Conv2d(cfg[2], planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * 4)
        self.select = channel_selection(planes*4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)
#         out = self.select(out)
        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class ResNet(Backbone):
    """ ResNet network module. Allows extracting specific feature blocks."""
    def __init__(self, block, layers, output_layers,cfg = None , num_classes=1000, inplanes=64, dilation_factor=1, frozen_layers=()):
        self.inplanes = inplanes
        super(ResNet, self).__init__(frozen_layers=frozen_layers)
        self.output_layers = output_layers
        self.conv1 = nn.Conv2d(3, inplanes , kernel_size=7, stride=2, padding=3,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(inplanes)

        self.relu = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        stride = [1 + (dilation_factor < l) for l in (8, 4, 2)]
        self.layer1 = self._make_layer(block, inplanes, layers[0], dilation=max(dilation_factor//8, 1), cfg = cfg[0:3*layers[0]])
        self.layer2 = self._make_layer(block, inplanes*2, layers[1], stride=stride[0], dilation=max(dilation_factor//4, 1), cfg = cfg[3*layers[0]:3*layers[1]+3*layers[0]])
        self.layer3 = self._make_layer(block, inplanes*4, layers[2], stride=stride[1], dilation=max(dilation_factor//2, 1), cfg = cfg[3*layers[1]+3*layers[0]:3*layers[1]+3*layers[0]+3*layers[2]])
        self.layer4 = self._make_layer(block, inplanes*8, layers[3], stride=stride[2], dilation=dilation_factor, cfg = cfg[3*layers[1]+3*layers[0]+3*layers[2]:3*layers[1]+3*layers[0]+3*layers[2]+3*layers[3]])

        out_feature_strides = {'conv1': 4, 'layer1': 4, 'layer2': 4*stride[0], 'layer3': 4*stride[0]*stride[1],
                               'layer4': 4*stride[0]*stride[1]*stride[2]}

        # TODO better way?
        if isinstance(self.layer1[0], BasicBlock):
            out_feature_channels = {'conv1': inplanes, 'layer1': inplanes, 'layer2': inplanes*2, 'layer3': inplanes*4,
                               'layer4': inplanes*8}
        elif isinstance(self.layer1[0], Bottleneck):
            base_num_channels = 4 * inplanes
            out_feature_channels = {'conv1': inplanes, 'layer1': base_num_channels, 'layer2': base_num_channels * 2,
                                    'layer3': base_num_channels * 4, 'layer4': base_num_channels * 8}
        else:
            raise Exception('block not supported')

        self._out_feature_strides = out_feature_strides
        self._out_feature_channels = out_feature_channels

        # self.avgpool = nn.AvgPool2d(7, stride=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(cfg[-1], num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def out_feature_strides(self, layer=None):
        if layer is None:
            return self._out_feature_strides
        else:
            return self._out_feature_strides[layer]

    def out_feature_channels(self, layer=None):
        if layer is None:
            return self._out_feature_channels
        else:
            return self._out_feature_channels[layer]

    def _make_layer(self, block, planes, blocks, stride=1, dilation=1, cfg=None):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
                channel_selection(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, cfg[0:3], stride, downsample, dilation=dilation))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, cfg[3*i:3*(i+1)]))

        return nn.Sequential(*layers)

    def _add_output_and_check(self, name, x, outputs, output_layers):
        if name in output_layers:
            outputs[name] = x
        return len(output_layers) == len(outputs)

    def forward(self, x, output_layers=None):
        """ Forward pass with input x. The output_layers specify the feature blocks which must be returned """
        outputs = OrderedDict()

        if output_layers is None:
            output_layers = self.output_layers

        x = self.conv1(x)
        x = self.bn1(x) ####### select daal dena
        x = self.relu(x)

        if self._add_output_and_check('conv1', x, outputs, output_layers):
            return outputs

        x = self.maxpool(x)

        x = self.layer1(x)

        if self._add_output_and_check('layer1', x, outputs, output_layers):
            return outputs

        x = self.layer2(x)

        if self._add_output_and_check('layer2', x, outputs, output_layers):
            return outputs

        x = self.layer3(x)

        if self._add_output_and_check('layer3', x, outputs, output_layers):
            return outputs

        x = self.layer4(x)

        if self._add_output_and_check('layer4', x, outputs, output_layers):
            return outputs

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        if self._add_output_and_check('fc', x, outputs, output_layers):
            return outputs

        if len(output_layers) == 1 and output_layers[0] == 'default':
            return x

        raise ValueError('output_layer is wrong.')


def resnet_baby(output_layers=None, pretrained=False, inplanes=16, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, inplanes=inplanes, **kwargs)

    if pretrained:
        raise NotImplementedError
    return model


def resnet18(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(BasicBlock, [2, 2, 2, 2], output_layers, **kwargs)

    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model


def resnet50(output_layers=None, pretrained=False,cfg = None,**kwargs):
    """Constructs a ResNet-50 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))
    model = ResNet(Bottleneck, [3, 4, 6, 3],output_layers,cfg=cfg,**kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50'], progress = False), strict = False )
    return model

def resnet101(output_layers=None, pretrained=False, **kwargs):
    """Constructs a ResNet-50 model.
    """

    if output_layers is None:
        output_layers = ['default']
    else:
        for l in output_layers:
            if l not in ['conv1', 'layer1', 'layer2', 'layer3', 'layer4', 'fc']:
                raise ValueError('Unknown layer: {}'.format(l))

    model = ResNet(Bottleneck,[3, 4, 23, 3], output_layers, **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model

In [14]:
cfg.pop(1)

'M'

In [15]:
cfg

[64,
 47,
 47,
 191,
 47,
 47,
 191,
 47,
 47,
 191,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 512,
 512,
 2048,
 512,
 512,
 2048,
 512,
 512,
 2048]

In [16]:
model_5 = resnet101(cfg=cfg)

In [17]:
model_5

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 47, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(47, 47, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(47, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (select): channel_selection()
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256

In [18]:
import os
newmodel = resnet101(cfg=cfg)
old_modules = list(model.modules())
new_modules = list(newmodel.modules())
layer_id_in_cfg = 0
start_mask = torch.ones(3)
end_mask = cfg_mask[layer_id_in_cfg]
conv_count = 0
first_bn = False

for layer_id in range(len(old_modules)):
    m0 = old_modules[layer_id]
    m1 = new_modules[layer_id]
    if isinstance(m0, nn.BatchNorm2d):
        if first_bn==False:
            first_bn=True
            m1.weight.data = m0.weight.data.clone()
            m1.bias.data = m0.bias.data.clone()
            m1.running_mean = m0.running_mean.clone()
            m1.running_var = m0.running_var.clone()
            layer_id_in_cfg += 1
            start_mask = end_mask.clone()
            if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
                end_mask = cfg_mask[layer_id_in_cfg]
            continue
        idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
        if idx1.size == 1:
            idx1 = np.resize(idx1,(1,))

        if isinstance(old_modules[layer_id + 1], channel_selection):
            # If the next layer is the channel selection layer, then the current batchnorm 2d layer won't be pruned.
            m1.weight.data = m0.weight.data.clone()
            m1.bias.data = m0.bias.data.clone()
            m1.running_mean = m0.running_mean.clone()
            m1.running_var = m0.running_var.clone()

            # We need to set the channel selection layer.
            m2 = new_modules[layer_id + 1]
            m2.indexes.data.zero_()
            m2.indexes.data[idx1.tolist()] = 1.0

            layer_id_in_cfg += 1
            start_mask = end_mask.clone()
            if layer_id_in_cfg < len(cfg_mask):
                end_mask = cfg_mask[layer_id_in_cfg]
        else:
            m1.weight.data = m0.weight.data[idx1.tolist()].clone()
            m1.bias.data = m0.bias.data[idx1.tolist()].clone()
            m1.running_mean = m0.running_mean[idx1.tolist()].clone()
            m1.running_var = m0.running_var[idx1.tolist()].clone()
            layer_id_in_cfg += 1
            start_mask = end_mask.clone()
            if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
                end_mask = cfg_mask[layer_id_in_cfg]

    
    elif isinstance(m0, nn.Conv2d):
        if conv_count == 0:
            m1.weight.data = m0.weight.data.clone()
            conv_count += 1
            print("BAHAR ka conv", m1.weight.data.shape)
            continue
        if isinstance(old_modules[layer_id+2], nn.Conv2d) and not isinstance(old_modules[layer_id-1], nn.BatchNorm2d):
            # Residual first case
            conv_count += 1
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
#             print('In shape: {:d}, Out shape {:d}**.'.format(idx0.size, idx1.size))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            if idx1.size == 1:
                idx1 = np.resize(idx1, (1,))
#             w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
            w1 = m0.weight.data[idx1.tolist(), :, :, :].clone()
            m1.weight.data = w1.clone()
            print("CASE 1 ", m1.weight.data.shape)
#             if conv_count % 3 != 1:           
            continue
        if isinstance(old_modules[layer_id+2], nn.Conv2d) and isinstance(old_modules[layer_id-1], nn.BatchNorm2d):
            # Residual second case
            conv_count += 1
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
#             print('In shape: {:d}, Out shape {:d}**.'.format(idx0.size, idx1.size))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            if idx1.size == 1:
                idx1 = np.resize(idx1, (1,))
            w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
            w1 = w1[idx1.tolist(), :, :, :].clone()
            m1.weight.data = w1.clone()
            print("CASE 2 ", m1.weight.data.shape)
#             if conv_count % 3 != 1:           
            continue
        if isinstance(old_modules[layer_id+3], nn.ReLU) and isinstance(old_modules[layer_id+2], channel_selection):
            # Residual third case
            conv_count += 1
            idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
            idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
#             print('In shape: {:d}, Out shape {:d}.'.format(idx0.size, idx1.size))
            if idx0.size == 1:
                idx0 = np.resize(idx0, (1,))
            if idx1.size == 1:
                idx1 = np.resize(idx1, (1,))
            w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
            m1.weight.data = w1.clone()
            print("CASE 3 ", m1.weight.data.shape)
            continue
        m1.weight.data = m0.weight.data.clone()
        print("DOWN ka conv", m1.weight.data.shape)
 
    elif isinstance(m0, nn.Linear):
        idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
        if idx0.size == 1:
            idx0 = np.resize(idx0, (1,))

        m1.weight.data = m0.weight.data[:, idx0].clone()
        m1.bias.data = m0.bias.data.clone()

BAHAR ka conv torch.Size([64, 3, 7, 7])
CASE 1  torch.Size([47, 64, 1, 1])
CASE 2  torch.Size([47, 47, 3, 3])
CASE 3  torch.Size([256, 47, 1, 1])
DOWN ka conv torch.Size([256, 64, 1, 1])
CASE 1  torch.Size([47, 256, 1, 1])
CASE 2  torch.Size([47, 47, 3, 3])
CASE 3  torch.Size([256, 47, 1, 1])
CASE 1  torch.Size([47, 256, 1, 1])
CASE 2  torch.Size([47, 47, 3, 3])
CASE 3  torch.Size([256, 47, 1, 1])
CASE 1  torch.Size([95, 256, 1, 1])
CASE 2  torch.Size([95, 95, 3, 3])
CASE 3  torch.Size([512, 95, 1, 1])
DOWN ka conv torch.Size([512, 256, 1, 1])
CASE 1  torch.Size([95, 512, 1, 1])
CASE 2  torch.Size([95, 95, 3, 3])
CASE 3  torch.Size([512, 95, 1, 1])
CASE 1  torch.Size([95, 512, 1, 1])
CASE 2  torch.Size([95, 95, 3, 3])
CASE 3  torch.Size([512, 95, 1, 1])
CASE 1  torch.Size([95, 512, 1, 1])
CASE 2  torch.Size([95, 95, 3, 3])
CASE 3  torch.Size([512, 95, 1, 1])
CASE 1  torch.Size([127, 512, 1, 1])
CASE 2  torch.Size([127, 127, 3, 3])
CASE 3  torch.Size([1024, 127, 1, 1])
DOWN ka conv torc

In [ ]:
# import os
# newmodel = resnet101(cfg=cfg)
# old_modules = list(model.modules())
# new_modules = list(newmodel.modules())
# layer_id_in_cfg = 0
# start_mask = torch.ones(3)
# end_mask = cfg_mask[layer_id_in_cfg]
# conv_count = 0
# first_bn = False

# for layer_id in range(len(old_modules)):
#     m0 = old_modules[layer_id]
#     m1 = new_modules[layer_id]
#     if isinstance(m0, nn.BatchNorm2d):
#         if first_bn==False:
#             first_bn=True
#             m1.weight.data = m0.weight.data.clone()
#             m1.bias.data = m0.bias.data.clone()
#             m1.running_mean = m0.running_mean.clone()
#             m1.running_var = m0.running_var.clone()
#             layer_id_in_cfg += 1
#             start_mask = end_mask.clone()
#             if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
#                 end_mask = cfg_mask[layer_id_in_cfg]
#             continue
#         idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
#         if idx1.size == 1:
#             idx1 = np.resize(idx1,(1,))

#         if isinstance(old_modules[layer_id + 1], channel_selection):
#             # If the next layer is the channel selection layer, then the current batchnorm 2d layer won't be pruned.
#             m1.weight.data = m0.weight.data.clone()
#             m1.bias.data = m0.bias.data.clone()
#             m1.running_mean = m0.running_mean.clone()
#             m1.running_var = m0.running_var.clone()

#             # We need to set the channel selection layer.
#             m2 = new_modules[layer_id + 1]
#             m2.indexes.data.zero_()
#             m2.indexes.data[idx1.tolist()] = 1.0

#             layer_id_in_cfg += 1
#             start_mask = end_mask.clone()
#             if layer_id_in_cfg < len(cfg_mask):
#                 end_mask = cfg_mask[layer_id_in_cfg]
#         else:
#             m1.weight.data = m0.weight.data[idx1.tolist()].clone()
#             m1.bias.data = m0.bias.data[idx1.tolist()].clone()
#             m1.running_mean = m0.running_mean[idx1.tolist()].clone()
#             m1.running_var = m0.running_var[idx1.tolist()].clone()
#             layer_id_in_cfg += 1
#             start_mask = end_mask.clone()
#             if layer_id_in_cfg < len(cfg_mask):  # do not change in Final FC
#                 end_mask = cfg_mask[layer_id_in_cfg]

    
#     elif isinstance(m0, nn.Conv2d):
#         if conv_count == 0:
#             m1.weight.data = m0.weight.data.clone()
#             conv_count += 1
#             print("BAHAR ka conv", m1.weight.data.shape)
#             continue
#         if isinstance(old_modules[layer_id+2], nn.Conv2d) and not isinstance(old_modules[layer_id-1], nn.BatchNorm2d):
#             # Residual first case
#             conv_count += 1
#             idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
#             idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
# #             print('In shape: {:d}, Out shape {:d}**.'.format(idx0.size, idx1.size))
#             if idx0.size == 1:
#                 idx0 = np.resize(idx0, (1,))
#             if idx1.size == 1:
#                 idx1 = np.resize(idx1, (1,))
# #             w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
#             w1 = m0.weight.data[idx1.tolist(), :, :, :].clone()
#             m1.weight.data = w1.clone()
#             print("CASE 1 ", m1.weight.data.shape)
# #             if conv_count % 3 != 1:           
#             continue
#         if isinstance(old_modules[layer_id+2], nn.Conv2d) and isinstance(old_modules[layer_id-1], nn.BatchNorm2d):
#             # Residual second case
#             conv_count += 1
#             idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
#             idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
# #             print('In shape: {:d}, Out shape {:d}**.'.format(idx0.size, idx1.size))
#             if idx0.size == 1:
#                 idx0 = np.resize(idx0, (1,))
#             if idx1.size == 1:
#                 idx1 = np.resize(idx1, (1,))
#             w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
#             w1 = w1[idx1.tolist(), :, :, :].clone()
#             m1.weight.data = w1.clone()
#             print("CASE 2 ", m1.weight.data.shape)
# #             if conv_count % 3 != 1:           
#             continue
#         if isinstance(old_modules[layer_id+3], nn.ReLU) and isinstance(old_modules[layer_id+2], channel_selection):
#             # Residual third case
#             conv_count += 1
#             idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
#             idx1 = np.squeeze(np.argwhere(np.asarray(end_mask.cpu().numpy())))
# #             print('In shape: {:d}, Out shape {:d}.'.format(idx0.size, idx1.size))
#             if idx0.size == 1:
#                 idx0 = np.resize(idx0, (1,))
#             if idx1.size == 1:
#                 idx1 = np.resize(idx1, (1,))
#             w1 = m0.weight.data[:, idx0.tolist(), :, :].clone()
#             m1.weight.data = w1.clone()
#             print("CASE 3 ", m1.weight.data.shape)
#             continue
#         m1.weight.data = m0.weight.data.clone()
#         print("DOWN ka conv", m1.weight.data.shape)
 
#     elif isinstance(m0, nn.Linear):
#         idx0 = np.squeeze(np.argwhere(np.asarray(start_mask.cpu().numpy())))
#         if idx0.size == 1:
#             idx0 = np.resize(idx0, (1,))

#         m1.weight.data = m0.weight.data[:, idx0].clone()
#         m1.bias.data = m0.bias.data.clone()

In [ ]:
# torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()},'/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/layerwise_pruned_50p.pth.tar')

In [19]:
torch.save({'cfg': cfg, 'state_dict': newmodel.state_dict()},'/workspace/tracking_datasets/pruned_ckpts/dimp101_correct/modified_batchnorm_layerwise_pruned_50p.pth.tar')

In [20]:
newmodel.load_state_dict(torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp101_correct/modified_batchnorm_layerwise_pruned_50p.pth.tar')['state_dict'])

<All keys matched successfully>

In [ ]:
# torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/layerwise_pruned_75p.pth.tar')['state_dict']['layer1.1.conv1.weight'].shape

In [ ]:
# torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/layerwise_pruned_75p.pth.tar')['state_dict']['layer1.1.conv2.weight'].shape

In [ ]:
# torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp50_correct/layerwise_pruned_75p.pth.tar')['state_dict']['layer1.1.conv3.weight'].shape

In [21]:
cfg1 = torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp101_correct/modified_batchnorm_layerwise_pruned_50p.pth.tar')['cfg']

In [22]:
cfg1

[64,
 47,
 47,
 191,
 47,
 47,
 191,
 47,
 47,
 191,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 512,
 512,
 2048,
 512,
 512,
 2048,
 512,
 512,
 2048]

In [23]:
newmodel = resnet101(cfg = cfg1)
newmodel

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 47, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(47, 47, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(47, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(47, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (select): channel_selection()
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256

In [24]:
cfg1

[64,
 47,
 47,
 191,
 47,
 47,
 191,
 47,
 47,
 191,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 95,
 95,
 383,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 127,
 127,
 511,
 512,
 512,
 2048,
 512,
 512,
 2048,
 512,
 512,
 2048]

In [25]:
new = resnet101(cfg = cfg1)
new.load_state_dict(torch.load('/workspace/tracking_datasets/pruned_ckpts/dimp101_correct/modified_batchnorm_layerwise_pruned_50p.pth.tar')['state_dict'])

<All keys matched successfully>

In [ ]:
new

In [ ]:
ckpt = torch.load('pruned.pth.tar')['state_dict']

In [ ]:
ckpt['conv1.weight'].shape

In [ ]:
conv
bn

layer1
    3conv 3bn
    X3
layer2
    3conv 3bn
    X4
layer3
    3conv 3bn
    X6
layer4
    3conv 3bn
    X3



In [ ]:
new

In [ ]:
newmodel